In [15]:
import cv2
import numpy as np
from pylab import imshow, show
import pandas as pd
import random
import os
import matplotlib.pyplot as plt
import math

output = "/Users/mcarcel/Desktop/test/results"
work_dir="/Users/mcarcel/Desktop/test/"
masks="/Users/mcarcel/Desktop/test/results/masks/"
script_file = work_dir+"/pos_5706_2.txt" #edit position list script


def parsePositionList(input_file):
    """ Recover positions coordinates from input file in an ordered list. 
    This version considers the pos values to be 
    ordered in the input file, returns an error if not. 
    This case needs to be handled if necessary.
    USAGE:
    file = "/Users/mcarcel/Downloads/pos_5706_2.txt"
    res = parsePositionList(file)
    if res != []:
        print(res)
    """
    
    positions = []
    current_pos = -1.
    current_Z = 0.
    current_Y = 0.
    current_X = 0.
    stage="INIT" #Init state: skip all entries until "LABEL" keyword is found 
    #stage cycle according to script syntax: 'INIT' -> WAITING' -> 'Z' -> 'Y' -> 'X' -> 'WAITING'
    for line in open(input_file,'r', encoding="ISO-8859-1"):
        line = line.split('\"') # split string to find keyword
        #print(line)
        if line[1%len(line)] == "LABEL":
            if stage == "INIT" or stage == "WAITING":
                #Reset values
                current_Z = 0.
                current_Y = 0.
                current_X = 0.
                #Update pos value
                current_pos = int(line[3].split('s')[1])
                #print("Found pos "+ str(current_pos))
                if current_pos == len(positions): # Pos values are ordered
                    stage="Z" #Go to next stage
                else:
                    print("Error: positions are not ordered in file. This case needs to be handled.")
                    return []
            else:
                print("Stage mismatch ERROR 1. Current stage: "+stage+". Please investigate")
                return []
        if line[1%len(line)] == "X":
            if stage == "Z":
                #Update Z value
                current_Z = float(line[2].split(',')[0].split(" ")[1])
                #print("Detected Z: "+str(current_Z))
                stage="Y" #Go to next stage
            elif stage == "X":
                #Update X value
                current_X = float(line[2].split(',')[0].split(" ")[1])
                #print("Detected X: "+str(current_X))
                #End of stage loop, save values and go back to first stage
                positions.append([current_pos,current_X,current_Y,current_Z])
                stage="WAITING" 
            elif stage == "INIT":
                #Skip value
                continue
            else :
                print("Stage mismatch ERROR 2. Current stage: "+stage+". Please investigate")
                return []
        if line[1%len(line)] == "Y":  
            if stage == "Y":
                #Update Y value
                current_Y = float(line[2].split(',')[0].split(" ")[1])
                #print("Detected Y: "+str(current_Y))
                stage="X" #Go to next stage
            elif stage == "Z" or stage == "INIT":
                #Skip value
                continue
            else:
                print("Stage mismatch ERROR 3. Current stage: "+stage+". Please investigate")
                return []
    return positions
    
def bytescaling(data, cmin=None, cmax=None, high=255, low=0):
    """
    Converting the input image to uint8 dtype and scaling
    the range to ``(low, high)`` (default 0-255). If the input image already has 
    dtype uint8, no scaling is done.
    :param data: 16-bit image data array
    :param cmin: bias scaling of small values (def: data.min())
    :param cmax: bias scaling of large values (def: data.max())
    :param high: scale max value to high. (def: 255)
    :param low: scale min value to low. (def: 0)
    :return: 8-bit image data array
    """
    if data.dtype == np.uint8:
        return data

    if high > 255:
        high = 255
    if low < 0:
        low = 0
    if high < low:
        raise ValueError("`high` should be greater than or equal to `low`.")

    if cmin is None:
        cmin = data.min()
    if cmax is None:
        cmax = data.max()

    cscale = cmax - cmin
    if cscale == 0:
        cscale = 1

    scale = float(high - low) / cscale
    bytedata = (data - cmin) * scale + low
    return (bytedata.clip(low, high) + 0.5).astype(np.uint8)

def compute_crop_mask(img, abscisse_centre, ordonnee_centre, radius_centre):

    # Compute masked blurred img
    masked_image = img.copy()
    rectangle = cv2.rectangle(masked_image, (math.floor(abscisse_centre) - math.floor(radius_centre), math.floor(ordonnee_centre) + math.floor(radius_centre)), (math.floor(abscisse_centre) + math.floor(radius_centre), math.floor(ordonnee_centre) - math.floor(radius_centre)),(0,0,0),0 )
    crop = rectangle[math.floor(ordonnee_centre) - math.floor(radius_centre) + 1 :math.floor(ordonnee_centre) + math.floor(radius_centre), math.floor(abscisse_centre) - math.floor(radius_centre) +1 :math.floor(abscisse_centre) + math.floor(radius_centre)]

    return crop

In [16]:
#### PARAMETERS ######
min_rayon = 100 #125
max_rayon = 200 #160
forced_radius = 150
param0 = 500 # Minimum dist between circle centers
param1 = 200 # threshold : first param of hough circle detect
param2 = 20 # the bigger the less circles are detected
radius_scale = 0.8 # scaling on the radii of the circles. eg: 0.8 means circles will be 20% smaller than detected
min_cc_size = 500 #min px
max_cc_size = 2000 #max px
time_window_size = 6
detection_rate = 0.8 #TODO : percentage of detection in time window to be considered positive (UNUSED NOW)

limit = 3 # 575 # number of folders to process
limit_frame = 8
first_well_of_position = 0

######################
print("Recovering positions and respective coordinates from script.")
position_coordinates = parsePositionList(script_file)
if res == []:
    print("Error, something happened during position coordinates extraction. Please investigate.")
else :
    print("OK")


print("Starting 8 bits scaling")
#Convert 16 bits to 8 bits + optimize range
for champ in range(0,limit):
    for frame in range (0,limit_frame) :
        compteur = 0
        root_img = "/Users/mcarcel/Desktop/test/Pos"+ str(champ)+"/img_"+ "{0:09d}".format(frame)+"_Default0_000.tif"
        img1 = cv2.imread(root_img, cv2.IMREAD_UNCHANGED | cv2.IMREAD_ANYCOLOR )

        img = bytescaling(img1) 

        cv2.imwrite("/Users/mcarcel/Desktop/test/Pos"+ str(champ)+"/img_"+ "{0:09d}".format(frame)+"_Default0_000.tif",img)
print("Scaling done! Starting well detection...")


# masks + mask coordinates
for champ in range(0,limit): #Loop on positions

    root_img =  "/Users/mcarcel/Desktop/test/Pos"+ str(champ)+"/img_000000000_Default0_000.tif"
    img = cv2.imread(root_img, cv2.IMREAD_UNCHANGED | cv2.IMREAD_ANYCOLOR )
    blur_img = cv2.blur(img, (16,16)) # blur input image
    
    height,width = img.shape

    mask = np.zeros((height,width),np.uint8)
    
    edges = cv2.Canny(blur_img, 10, 10) # Canny edge detection
    scaled_edges = bytescaling(edges) # optimize range if needed
    
    circles = cv2.HoughCircles(scaled_edges, cv2.HOUGH_GRADIENT, 1, param0, param1 = param1, param2 = param2, minRadius = min_rayon, maxRadius = max_rayon)
    a = []
    o = []
    d = []
    good_circles = []
    circle_img = np.zeros((height,width),np.uint8)
    if circles is not None :
        avg_radius = 0
        count = 0
        for i in circles[0] : # Compute average circle radius
            avg_radius += i[2] # avg_radius = avg_radius + i[2]
            count += 1
        
        if count == 0 :
            print("Error: this should not happen. Please investigate.")
            
        avg_radius /= count
        avg_radius = int(avg_radius*radius_scale)
        for i in circles[0] :
            if (i[0] > min_rayon and i[0] < width - min_rayon ) and (i[1] > min_rayon and i[1] < height - min_rayon ) :                
                # Draw on mask
                cv2.circle(mask,(i[0],i[1]),avg_radius,(255,255,255),thickness = -1)
                good_circles.append(i)
                
                a.append(i[0])
                o.append(i[1])

                d.append(forced_radius)

    Dataset = list(zip(a, o, d))

    last_well_of_position = first_well_of_position + len(Dataset)
    index_df = range(first_well_of_position, last_well_of_position)
    df = pd.DataFrame(data=Dataset, index = index_df, columns=['x', 'y', 'r'])
#                 df = df[df.x > max_rayon]
#                 df = df[df.x < width - max_rayon]
#                 df = df[df.y > max_rayon]
#                 df = df[df.y < height - max_rayon]
     
    masked_data = cv2.bitwise_and(img, img, mask=mask)
    
    font = cv2.FONT_HERSHEY_SIMPLEX
    for puits in range(first_well_of_position, last_well_of_position):
        write_wells = cv2.putText(masked_data, str(puits), (int(df.loc[puits][0]), int(df.loc[puits][1])), font, 5, (255, 255, 255), 10, cv2.LINE_AA)

                
    masked_img = img * (write_wells/255)
    #DEBUG OUTPUT#
    cv2.imwrite(output + "/masks" + '/masks_num_'+str(champ)+'.png',masked_img)
    ##############
    first_well_of_position=df.index[-1] + 1

    name_save = output + "/mask_coordinates_"+ str(champ) + ".csv"
    df.to_csv(name_save, index=True)
    
    print(str(champ+1) + "/" + str(limit) + " complete.")
    
print("Masks done. Cropping")

first_well_position = 0
for champ in range(0,limit): #Loop on pos
    print("Computing pos "+str(champ))

    for frame in range(0, limit_frame):
        print("Computing frame "+str(frame))
        #load image
        root_img = work_dir + "Pos" + str(champ) + "/img_" + "{0:09d}".format(frame) + "_Default0_000.tif"    
        img = cv2.imread(root_img, cv2.IMREAD_UNCHANGED | cv2.IMREAD_ANYCOLOR )
        # recover detected circles values from csv file
        circles = pd.read_csv(output + "/mask_coordinates_"+ str(champ) + ".csv", index_col = 0)
        
        last_well_position = first_well_position + len(circles)

        
        for row in range(first_well_position, last_well_position): #Loop on wells

            #Init mask
            mask = np.zeros_like(img)
            #compute mask
            cv2.circle(mask,(math.floor(circles.loc[row][0]),math.floor(circles.loc[row][1])),(math.floor(circles.loc[row][2])),(1,1,1),thickness = -1)

            #Compute crop
            well = compute_crop_mask(img, math.floor(circles.loc[row][0]), math.floor(circles.loc[row][1]), math.floor(circles.loc[row][2]))
            
            root_save = '/Users/mcarcel/Desktop/test/wells/well_' + str(row)
            os.makedirs(root_save, exist_ok = True)
            name_save = root_save + '/well_' + str(row) + '_frame_'+ str(frame) + '.tif'
            
            #Write cropped well
            cv2.imwrite(name_save, well)
            
            #####  Metadata Management ####
            #Load metadata template
            if frame == 0:
                metadata_template_file = open(work_dir+"/metadata_template.txt","r", encoding="ISO-8859-1")
                metadata_content = metadata_template_file.read()
                metadata_template_file.close()
                pos_coords = position_coordinates[champ]
                if pos_coords[0] != champ:
                    print("Pos index mismatch error.")
                metadata_content+= "\n"+"MicroWell number: "+str(row)+"\n"
                metadata_content+="Abscissa :"+str(math.floor(circles.loc[row][0]))+"\n"
                metadata_content+="Ordinate : "+str(math.floor(circles.loc[row][1]))+"\n"
                metadata_content+= "\n"
                metadata_content+="X:"+ str(pos_coords[1])+"\n"
                metadata_content+="Y:"+ str(pos_coords[2])+"\n"
                metadata_content+="Z:"+ str(pos_coords[3])+"\n"
                #Write metadata
                output_metadata = open(root_save + '/well_' + str(row) + '_metadata.txt','w')
                output_metadata.write(metadata_content)
                output_metadata.close()
            ###############################
    first_well_position = (circles.index[-1]) + 1

print ('Cropping done!')


Recovering positions and respective coordinates from script.
OK
Starting 8 bits scaling
Scaling done! Starting well detection...
1/3 complete.
2/3 complete.
3/3 complete.
Masks done. Cropping
Computing pos 0
Computing frame 0
Computing frame 1
Computing frame 2
Computing frame 3
Computing frame 4
Computing frame 5
Computing frame 6
Computing frame 7
Computing pos 1
Computing frame 0
Computing frame 1
Computing frame 2
Computing frame 3
Computing frame 4
Computing frame 5
Computing frame 6
Computing frame 7
Computing pos 2
Computing frame 0
Computing frame 1
Computing frame 2
Computing frame 3
Computing frame 4
Computing frame 5
Computing frame 6
Computing frame 7
Cropping done!
